In [2]:
import requests
import pandas as pd
import time
import random
import csv
from urllib.request import urlopen
import json
from itertools import islice
from urllib.error import URLError, HTTPError

class CategoriesLinksCrawling:
    def __init__(self):
        self.categories_file_path = './data/category_ids.csv'
        self.special_link_file_path = './data/categories_special_link_v2.csv'
        self.base_link = 'https://tiki.vn'
        self.api_menu = 'https://api.tiki.vn/raiden/v2/menu-config?'
        self.api_sub_menu = 'https://tiki.vn/api/personalish/v1/blocks/listings?'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
            'Accept': 'application/json, text/plain, */*',
            'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7',
        #     'Referer': 'https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html?src=category-page-1789&2hi=0',
            'x-guest-token': 'noUm6Bg7hPXcdYtiHREzDfxIOClk1Nup',
            'Connection': 'keep-alive',
            'TE': 'Trailers',
        }
        self.params = {
            'platform': 'desktop'
        }
        
        self.params_api = {
            'limit':'40',
            'include':'advertisement',
            'aggregations':'2',
            'trackity_id':'5a387aad-bb84-2dca-2df9-946cc84ccf0f',
            'page':'1'
        }
    def store_category_ids(self, filePath, url, category_id):
        with open(filePath, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)

            # Kiểm tra nếu file CSV rỗng, thêm header
            if csvfile.tell() == 0:
                writer.writerow(['url', 'category'])
            writer.writerow([url, category_id])
    
    def menu_craw(self):
        response = requests.get(self.api_menu, headers=self.headers, params=self.params)
        if response is not None:
            data = response.json()
            items = data.get("menu_block","").get("items", "")
            if items:
                for item in items:
                    link = item.get("link").split("/")
                    url = link[-2]
                    category_id = link[-1].split('c')[-1]
                    self.sub_menu_craw(url, category_id)
                    self.store_category_ids(self.categories_file_path, url, category_id)
                    
    def sub_menu_craw(self, url, category_id):
        self.params_api['category'] = category_id
        self.params_api['url'] = url
        response = requests.get(self.api_sub_menu , headers=self.headers, params=self.params_api)
        if response:
            data = response.json()
            filters = data.get("filters", "")
            if filters:
                if filters[0].get("query_name", "") == "category":
                    items = filters[0].get("values", "")
                    for item in items:
                        url = item.get("url_key", "")
                        category_id = item.get("query_value", "")
                        time.sleep(1)
                        self.sub_menu_craw(url, category_id)
                        self.store_category_ids(self.categories_file_path, url, category_id)
        
    def main(self):
        self.menu_craw()
        print("Successed")

if __name__ == "__main__":
    crawler = CategoriesLinksCrawling()
    crawler.main()

Successed
